In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())
words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ","]

In [3]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])
            
print(documents)

[(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi'], 'greetings'), (['good', 'day'], 'greetings'), (['Greetings'], 'greetings'), (['what', "'s", 'up', '?'], 'greetings'), (['how', 'is', 'it', 'going', '?'], 'greetings'), (['cya'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['bye'], 'goodbye'), (['cao'], 'goodbye'), (['see', 'ya'], 'goodbye'), (['how', 'old'], 'age'), (['how', 'old', 'is', 'Hasan'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['how', 'old', 'are', 'you'], 'age'), (['age', '?'], 'age'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'I', 'call', 'you'], 'name'), (['whats', 'your', 'name', '?'], 'name'), (['who', 'are', 'you', '?'], 'name'), (['Can', 'you', 'tell', 'me', 'your', 'name', '?'], 'name'), (['Id', 'like', 'to', 'buy', 'something'], 'shop'), (['what', 'are', 'your', 'products', '?'], 'shop'), (['what', 'do', 'you', 'recomm

In [4]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
print(words)

["'s", 'Can', 'Good', 'Goodbye', 'Greetings', 'Hasan', 'Have', 'I', 'Id', 'Leaving', 'See', 'What', 'a', 'age', 'am', 'are', 'buy', 'bye', 'call', 'cao', 'close', 'cya', 'day', 'do', 'going', 'good', 'guy', 'hello', 'hey', 'hi', 'hour', 'how', 'is', 'it', 'later', 'like', 'me', 'name', 'old', 'open', 'opening', 'product', 'recommend', 'see', 'sell', 'selling', 'should', 'something', 'tell', 'time', 'to', 'up', 'what', 'whats', 'when', 'who', 'ya', 'you', 'your']


In [5]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [6]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [7]:
random.shuffle(training)
training = np.array(training)

X_train = list(training[:, 0])
y_train = list(training[:, 1])

C:\Users\hasan\AppData\Local\Temp/ipykernel_11752/633689806.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation="softmax"))

adam = Adam(learning_rate=0.001, decay=1e-6)

model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])
history = model.fit(np.array(X_train), np.array(y_train), 
                    epochs=200, batch_size=16, verbose=1)

Epoch 1/200
3/3 [==============================] - 1s 5ms/step - loss: 1.8303 - accuracy: 0.2000
Epoch 2/200
3/3 [==============================] - 0s 5ms/step - loss: 1.8105 - accuracy: 0.2000
Epoch 3/200
3/3 [==============================] - 0s 6ms/step - loss: 1.7687 - accuracy: 0.2286
Epoch 4/200
3/3 [==============================] - 0s 4ms/step - loss: 1.7692 - accuracy: 0.2286
Epoch 5/200
3/3 [==============================] - 0s 6ms/step - loss: 1.7537 - accuracy: 0.2286
Epoch 6/200
3/3 [==============================] - 0s 5ms/step - loss: 1.8071 - accuracy: 0.2286
Epoch 7/200
3/3 [==============================] - 0s 5ms/step - loss: 1.7164 - accuracy: 0.3143
Epoch 8/200
3/3 [==============================] - 0s 5ms/step - loss: 1.6851 - accuracy: 0.2571
Epoch 9/200
3/3 [==============================] - 0s 5ms/step - loss: 1.6952 - accuracy: 0.3429
Epoch 10/200
3/3 [==============================] - 0s 5ms/step - loss: 1.6945 - accuracy: 0.3143
Epoch 11/200
3/3 [===========

In [9]:
model.save("chatbot_model.h5", history)